In [1]:
import pulp

problem = pulp.LpProblem("SLE", pulp.LpMaximize)

x = pulp.LpVariable("x", cat="Continuous")
y = pulp.LpVariable("y", cat="Continuous")

problem += 120 * x + 150 * y == 1440
problem += x + y == 10

status = problem.solve()

print("Status:", pulp.LpStatus[status])
print("x=", x.value(), "y=", y.value())

Status: Optimal
x= 2.0 y= 8.0


In [2]:
import pulp

problem = pulp.LpProblem("LP", pulp.LpMaximize)

x = pulp.LpVariable("x", cat="Continuous")
y = pulp.LpVariable("y", cat="Continuous")

problem += 1 * x + 3 * y <= 30
problem += 2 * x + 1 * y <= 40
problem += x >= 0
problem += y >= 0
problem += x + 2 * y

status = problem.solve()

print("Status:", pulp.LpStatus[status])
print("x=", x.value(), "y=", y.value(), "obj=", problem.objective.value())

Status: Optimal
x= 18.0 y= 4.0 obj= 26.0


In [3]:
import pandas as pd
import pulp

stock_df = pd.read_csv("stocks.csv")
stock_df

,m,stock
0,m1,35
1,m2,22
2,m3,27


In [4]:
require_df = pd.read_csv("requires.csv")
require_df

,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2
5,p2,m3,0
6,p3,m1,0
7,p3,m2,2
8,p3,m3,2
9,p4,m1,2


In [6]:
gain_df = pd.read_csv("gains.csv")
gain_df

,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


In [7]:
P = gain_df["p"].tolist()
print(P)

['p1', 'p2', 'p3', 'p4']


In [8]:
M = stock_df["m"].tolist()
print(M)

['m1', 'm2', 'm3']


In [10]:
stock = dict(zip(stock_df["m"], stock_df["stock"]))
print(stock)

{'m1': 35, 'm2': 22, 'm3': 27}


In [13]:
require = {(row.p, row.m): row.require for row in require_df.itertuples()}
print(require)

{('p1', 'm1'): 2, ('p1', 'm2'): 0, ('p1', 'm3'): 1, ('p2', 'm1'): 3, ('p2', 'm2'): 2, ('p2', 'm3'): 0, ('p3', 'm1'): 0, ('p3', 'm2'): 2, ('p3', 'm3'): 2, ('p4', 'm1'): 2, ('p4', 'm2'): 2, ('p4', 'm3'): 2}


In [15]:
gain = {row.p: row.gain for row in gain_df.itertuples()}
print(gain)

{'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}


In [17]:
problem = pulp.LpProblem("LP2", pulp.LpMaximize)

x = pulp.LpVariable.dicts("x", P, cat="Continuous")

for p in P:
    problem += x[p] >= 0

for m in M:
    problem += pulp.lpSum([require[p, m] * x[p] for p in P]) <= stock[m]

problem += pulp.lpSum([gain[p] * x[p] for p in P])

status = problem.solve()

print("Status:", pulp.LpStatus[status])
for p in P:
    print("x[", p, "]=", x[p].value())
print("obj=", problem.objective.value())

Status: Optimal
x[ p1 ]= 12.142857
x[ p2 ]= 3.5714286
x[ p3 ]= 7.4285714
x[ p4 ]= 0.0
obj= 80.42857099999999


In [ ]:
import pandas as pd
import pulp

# データの取得
require_df = pd.read_csv("requires.csv")
stock_df = pd.read_csv("stocks.csv")
gain_df = pd.read_csv("gains.csv")

# 集合の定義
P = gain_df["p"].tolist()
M = stock_df["m"].tolist()

# 定数の定義
stock = {row.m: row.stock for row in stock_df.itertuples()}
gain = {row.p: row.gain for row in gain_df.itertuples()}
require = {(row.p, row.m): row.require for row in require_df.itertuples()}

# 数理最適化モデルの定義
problem = pulp.LpProblem("IP", pulp.LpMaximize)  # 変更点（任意）

# 変数の定義
x = pulp.LpVariable.dicts("x", P, cat="Integer")  # 変更点

# 制約式の定義
for p in P:
    problem += x[p] >= 0
for m in M:
    problem += pulp.lpSum([require[p, m] * x[p] for p in P]) <= stock[m]

# 目的関数の定義
problem += pulp.lpSum([gain[p] * x[p] for p in P])

# 求解
status = problem.solve()
print("Status:", pulp.LpStatus[status])

# 計算結果の表示
for p in P:
    print(p, x[p].value())

print("obj=", problem.objective.value())